In [1]:
import nltk
from nltk.corpus import stopwords
import string
import numpy as np
from collections import Counter
from collections import defaultdict

In [2]:
#nltk.download()

In [3]:
words=nltk.corpus.brown.words()

In [4]:
len(words)

1161192

In [5]:
list_stopWords=list(set(stopwords.words('english')))

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
filtered_words=[str(w).lower() for w in words[:] if not (str(w).lower() in list_stopWords or str(w) in string.punctuation or str(w)=='``'or str(w)=="''")]

In [8]:
len(filtered_words)

541723

In [9]:
filtered_words=[str(w).lower() for w in filtered_words if not (w in list_stopWords or w=='--')]

In [10]:
len(filtered_words)

538291

In [11]:
counter=Counter(filtered_words)

In [12]:
len(counter)

49637

In [14]:
C=counter.most_common(1000)

In [15]:
Cset=set([c[0] for c in C])

In [16]:
V=counter.most_common(5000)

In [17]:
Vset=set([v[0] for v in V])

In [86]:
Vlist=[v[0] for v in V]

# n(w, c) = # of times c occurs in a window around w.

In [20]:
V[:30]

[('one', 3292),
 ('would', 2714),
 ('said', 1961),
 ('new', 1635),
 ('could', 1601),
 ('time', 1598),
 ('two', 1412),
 ('may', 1402),
 ('first', 1361),
 ('like', 1292),
 ('man', 1207),
 ('even', 1170),
 ('made', 1125),
 ('also', 1069),
 ('many', 1030),
 ('must', 1013),
 ('af', 996),
 ('back', 966),
 ('years', 950),
 ('much', 937),
 ('way', 908),
 ('well', 897),
 ('people', 847),
 ('mr.', 844),
 ('little', 831),
 ('state', 807),
 ('good', 806),
 ('make', 794),
 ('world', 787),
 ('still', 782)]

In [22]:
nWC=defaultdict(int)
for w in range(len(filtered_words)):
    if filtered_words[w] in Vset:
        if w-2>=0:
            if filtered_words[w-2] in Cset:
                nWC[(filtered_words[w],filtered_words[w-2])]+=1
        if w-1>=0:
            if filtered_words[w-1] in Cset:
                nWC[(filtered_words[w],filtered_words[w-1])]+=1
        if w+1<len(filtered_words):
            if filtered_words[w+1] in Cset:
                nWC[(filtered_words[w],filtered_words[w+1])]+=1
        if w+2<len(filtered_words):
            if filtered_words[w+2] in Cset:
                nWC[(filtered_words[w],filtered_words[w+2])]+=1

In [23]:
len(nWC)

466059

# Pr(C|W)

In [25]:
PrCW=np.zeros((1000,5000))

In [26]:
for c in range(1000):
    for w in range(5000):
        PrCW[c,w]=nWC[(V[w][0],C[c][0])]

In [27]:
tmp=np.sum(PrCW,axis=0)

In [29]:
PrCW=PrCW/tmp

# Pr(C)

In [32]:
PrC=np.zeros(1000)

In [33]:
totalc=sum([count[1] for count in C])

In [34]:
for i in range(1000):
    PrC[i]=C[i][1]*1./totalc

# $\phi_c(w)$

In [35]:
PhiCW=np.zeros((1000,5000))

In [43]:
tmp=np.log(PrCW/PrC.reshape(-1,1))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [60]:
PhiCW=np.nan_to_num((tmp>0)*1*tmp)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


In [61]:
PhiCW

array([[-0.        ,  0.04589105,  0.03686131, ...,  0.        ,
         0.87171948,  0.        ],
       [ 0.04098438,  0.15672467,  0.51279582, ...,  0.        ,
         1.0647912 ,  0.91064052],
       [-0.        ,  0.46730495,  0.24124449, ...,  1.07738553,
         1.38976022,  0.        ],
       ...,
       [-0.        ,  0.35914367,  2.19594062, ...,  0.        ,
         0.        ,  0.        ],
       [-0.        ,  0.86996929,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [62]:
from sklearn.decomposition import PCA

In [65]:
pca=PCA(n_components=100)

In [70]:
pca.fit(PhiCW.T)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [73]:
embed=pca.fit_transform(PhiCW.T)

In [74]:
embed.shape

(5000, 100)

In [76]:
from sklearn.cluster import KMeans

In [77]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(embed)

In [83]:
Counter(kmeans.labels_)

Counter({0: 21,
         1: 9,
         2: 140,
         3: 31,
         4: 15,
         5: 71,
         6: 27,
         7: 65,
         8: 23,
         9: 89,
         10: 58,
         11: 36,
         12: 39,
         13: 23,
         14: 43,
         15: 57,
         16: 58,
         17: 54,
         18: 94,
         19: 32,
         20: 39,
         21: 47,
         22: 18,
         23: 69,
         24: 47,
         25: 70,
         26: 72,
         27: 70,
         28: 72,
         29: 81,
         30: 53,
         31: 40,
         32: 97,
         33: 46,
         34: 46,
         35: 66,
         36: 25,
         37: 58,
         38: 64,
         39: 24,
         40: 71,
         41: 41,
         42: 64,
         43: 35,
         44: 33,
         45: 30,
         46: 66,
         47: 48,
         48: 22,
         49: 46,
         50: 14,
         51: 95,
         52: 28,
         53: 95,
         54: 25,
         55: 36,
         56: 22,
         57: 67,
         58: 46,
       

In [95]:
Vlist.index('green'),Vlist.index('yellow'),Vlist.index('red')

(772, 1888, 368)

In [128]:
predkmeans=kmeans.predict(embed[:,:])

In [134]:
for i in range(len(predkmeans)):
    if predkmeans[i]==27:
        print Vlist[i],

white black red hair blue green deep mouth thin bright neck gray lips thick dress fingers wore holding shoulder nose leg pale hole sky beneath hat yellow tall wood grass chest shoulders throat tiny pair pink suit rolled skin wearing shoes dirt lifted golden burned knees stomach tongue bent jacket falling slipped lightly pressed bare pot powder shirt beard leather belly tie backward shining stiff blonde boots grabbed trim brushed


In [130]:
Vlist[predkmeans[0]]

'fact'

In [97]:
kmeans.predict(embed[368:368+1,:])

array([27], dtype=int32)

In [99]:
from sklearn.neighbors import NearestNeighbors

In [103]:
neigh = NearestNeighbors(metric='cosine',n_neighbors=2)

In [104]:
neigh.fit(embed)  

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [105]:
neigh.kneighbors(embed[772:773,:], return_distance=False)

array([[772, 576]])

In [106]:
Vlist[576]

'blue'

In [110]:
test=['also','like','first','one','green','communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', 'chicago', 'revolution', 'september', 'chemical', 'detergent', 'dictionary', 'storm', 'worship']

In [117]:
test=test+['way','little','us','go','see','never']

In [125]:
for i in test:
    ind=Vlist.index(i)
    pred=neigh.kneighbors(embed[ind:ind+1,:], return_distance=False)[0][1]
    print i,
    print '\t\t\t',Vlist[pred]

also 			although
like 			said
first 			last
one 			thing
green 			blue
communism 			recording
autumn 			storm
cigarette 			lighted
pulmonary 			artery
mankind 			world
africa 			asia
chicago 			club
revolution 			renaissance
september 			july
chemical 			instances
detergent 			tubes
dictionary 			text
storm 			autumn
worship 			conscience
way 			i'm
little 			woman
us 			let
go 			i'll
see 			tell
never 			knew
